In [1]:
# Training an FNO for the Darcy flow Problem

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import numpy as np
import scipy.io
import h5py
import torch.nn as nn
from sklearn.model_selection import train_test_split

import operator
from functools import reduce
from functools import partial

import torch.nn.functional as F

import matplotlib.pyplot as plt

from timeit import default_timer

from torch.optim import Adam

from FNO2D import *
from utilities3 import *

torch.manual_seed(0)
np.random.seed(0)
torch.set_printoptions(precision=8)

In [4]:
# load the data
ntrain = 800
nval = 200

s = 128
reader = MatReader('df_train_128.mat')
coeff = reader.read_field('coeff')
sol = reader.read_field('sol')


In [5]:
# split into train/ validation : 80 / 20
x_train, x_val, y_train, y_val = train_test_split(coeff, sol, random_state=0, test_size=0.20)

In [6]:
# hyperparameters # 

ntrain = 800
ntest = 200

batch_size = 20
learning_rate = 0.001

epochs = 100
step_size = 100
gamma = 0.5

modes = 12
width = 32


In [7]:
# data normalisation
x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_val = x_normalizer.encode(x_val)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
x_val = x_val.reshape(nval,s,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_val, y_val), batch_size=batch_size, shuffle=False)

In [ ]:
# training the FNO

model = FNO2d(modes, modes, width) #.cuda()

optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
t1 = default_timer()

# y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    train_l2 = 0
    
    for x, y in train_loader:
        # x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x).reshape(batch_size, s, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()

    scheduler.step()

    model.eval()
    val_l2 = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            # x, y = x.cuda(), y.cuda()

            out = model(x).reshape(batch_size, s, s)
            out = y_normalizer.decode(out)

            val_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

    train_l2/= ntrain
    val_l2 /= nval

    t2 = default_timer()
    print("Epoch:",ep,"Time taken:", t2-t1, "Training Loss:", train_l2, "Validation Loss:", val_l2)

In [ ]:


reader = MatReader('df_test_256.mat')
x_test_256 = reader.read_field('coeff')
x_normalizer_256 = UnitGaussianNormalizer(x_test_256)
y_test_256 = reader.read_field('sol')
y_normalizer_256 = UnitGaussianNormalizer(y_test_256)
x_test_256 = x_normalizer_256.encode(x_test_256)
x_test_256 = x_test_256.reshape(200,256,256,1)



reader.load_file('df_test_128.mat')
x_test_128 = reader.read_field('coeff')
x_normalizer_128 = UnitGaussianNormalizer(x_test_128)
y_test_128 = reader.read_field('sol')
y_normalizer_128 = UnitGaussianNormalizer(y_test_128)
x_test_128 = x_normalizer_128.encode(x_test_128)
x_test_128 = x_test_128.reshape(200,128,128,1)



reader.load_file('df_test_64.mat')
x_test_64 = reader.read_field('coeff')
x_normalizer_64 = UnitGaussianNormalizer(x_test_64)
y_test_64 = reader.read_field('sol')
y_normalizer_64 = UnitGaussianNormalizer(y_test_64)
x_test_64 = x_normalizer_64.encode(x_test_64)
x_test_64 = x_test_64.reshape(200,64,64,1)

test_loader_256 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_256, y_test_256), batch_size=1, shuffle=False)

test_loader_128 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_128, y_test_128), batch_size=1, shuffle=False)

test_loader_64 = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_64, y_test_64), batch_size=1, shuffle=False)



In [ ]:
mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_256:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 256, 256)
        out = y_normalizer_256.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
print("resolution: 256", "Time taken:", t2-t1, "Testing Loss:", test_l2)

In [ ]:
mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_128:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 128, 128)
        out = y_normalizer_128.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
print("resolution: 128", "Time taken:", t2-t1, "Testing Loss:", test_l2)

In [ ]:
mytestloss = LpLoss(size_average=False)
t1 = default_timer()

model.eval()
test_l2 = 0.0
with torch.no_grad():
    for x, y in test_loader_64:
                # x, y = x.cuda(), y.cuda()
            
        out = model(x).reshape(1, 64, 64)
        out = y_normalizer_64.decode(out)

        test_l2 += mytestloss(out.view(1,-1), y.view(1,-1)).item()


test_l2 /= 200

t2 = default_timer()
print("resolution: 64", "Time taken:", t2-t1, "Testing Loss:", test_l2)